# Incremental Ernollment Trial
---------

여러개의 초기 enrollment의 case도 고려해서 만드려고 한다.

그리고 session을 고려한 case를 만들고싶다.

## Environment

In [11]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

from utils import key2df

In [12]:
sys.path.append('/host/projects/sv_experiments/sv_system')

## Dataframes & embeddings

In [14]:
keys = pickle.load(open("./xvector_embeds/sv_keys.pkl", "rb"))
keys = np.array(keys)
embeds = np.load("./xvector_embeds/sv_embeds.npy")
embeds = embeds / np.linalg.norm(embeds, axis=1, keepdims=True)

In [15]:
key_df = key2df(keys)
key2index = {k:v for v, k in enumerate(keys)}  # it used for mapping from key to index

In [16]:
spk_uttr_stat = key_df.spk.value_counts()

## Held-out validation set

utterance가 150개 넘는 것들을 trial spks로 빼고  
나머지는 validation spk로 빼었다.  
그리고 validation spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [ ]:
## trial_enr_spks = spk_uttr_stat[spk_uttr_stat > 150].index.tolist()
val_spks = spk_uttr_stat[spk_uttr_stat <= 150].index.tolist()
val_embed_df = key_df[key_df.spk.isin(val_spks)]
trial_embed_df = key_df[key_df.spk.isin(trial_enr_spks)]

In [18]:
voxc2_trial = pd.read_pickle("trials/voxc2_1211_trials.pkl")
trial_for_thresh = voxc2_trial[(voxc2_trial.enroll_spk.isin(val_spks)) & (voxc2_trial.test_spk.isin(val_spks))]

In [63]:
os.makedirs("./cases/enr306_uttr3/")
trial_for_thresh.to_pickle("cases/enr306_uttr3/trial_for_thresh.pkl")
val_embed_df.to_pickle("./cases/enr306_uttr3/valdiation_df.pkl")

## Design trials for each enr_spk

각 enr_spk당 10번 enr_uttrs 

In [67]:
# for multiple enrollemnt
cases = []
n_enroll_utters = 3
for enr_spk in trial_enr_spks:
    target_uttrs = trial_embed_df[trial_embed_df.spk == enr_spk] 
    nonTarget_uttrs = trial_embed_df[trial_embed_df.spk != enr_spk]
    # we have target ratios [0.9, 0.5, 0.1, 0.01]
    nonTarget_n_uttrs = (np.array(len(target_uttrs)-n_enroll_utters)*np.array([1/9, 1, 9])).astype(np.int64)
    # for p_ratio is 0.01 
    nonTarget_n_uttrs = np.append(nonTarget_n_uttrs, min(len(nonTarget_uttrs), 99*len(target_uttrs)))
    enr_uttr_pool = target_uttrs.sample(n=5*n_enroll_utters) # each speaker has 10 enrollment cases
    for i in range(0, len(enr_uttr_pool), n_enroll_utters):
        enr_uttrs = enr_uttr_pool[i:i+n_enroll_utters]
        for n_imp_uttr in nonTarget_n_uttrs: # neg_trial ratio 
            target_trial_uttrs = target_uttrs.drop(index=enr_uttrs.index)
            nonTarget_trial_uttrs = nonTarget_uttrs.sample(n=n_imp_uttr)
            cases += [[enr_spk, 
                       enr_uttrs.index.tolist(), 
                       target_trial_uttrs.index.tolist(),
                       nonTarget_trial_uttrs.index.tolist()]]

In [68]:
sorted_cases = sorted(cases, key=lambda x: len(x[3])) 

## Split trials according to pos_ratio

In [69]:
neg_legnth = list(map(lambda x: len(x[3]), cases))

In [70]:
pos_ratio_0 = cases[0:len(cases):4] # 0.9
pos_ratio_1 = cases[1:len(cases):4] # 0.5
pos_ratio_2 = cases[2:len(cases):4] # 0.1
pos_ratio_3 = cases[3:len(cases):4] # 0.01

In [71]:
pickle.dump(pos_ratio_0, open("./trials/enr306_uttr3/trials_ratio_0.9.pkl", "wb"))
pickle.dump(pos_ratio_1, open("./trials/enr306_uttr3/trials_ratio_0.5.pkl", "wb"))
pickle.dump(pos_ratio_2, open("./trials/enr306_uttr3/trials_ratio_0.1.pkl", "wb"))
pickle.dump(pos_ratio_3, open("./trials/enr306_uttr3/trials_ratio_0.01.pkl", "wb"))

In [72]:
len(pos_ratio_1)

1530